In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, get_data_commitment_maps

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

Here is a weird example where different col1 and col2 of data.json have different row number. We just want to show that different dimension is possible. In fact, a person can just request median(col_1), and median(col_2). Then just compute mean on his own as well, but here we show that the code is composable enough to do all at once. 

In [5]:

data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')
create_dummy(data_path, dummy_data_path)


In [6]:
scales = [8]
selected_columns = ['col_1', 'col_2']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [7]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x1, x2 = data[0], data[1]
    out_0 = s.median(x1)
    out_1 = s.median(x2)
    return s.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))

error = 0.0001
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path, verifier_model, verifier_model_path)

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_43756/589984064.py:9: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return s.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_43756/589984064.py:9: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return s.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))
/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:166: TracerWarning: torch.tensor results are registered as constants in the trace. You ca

In [8]:
# prover calculates settings, send to verifier
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path,selected_columns, sel_data_path, prover_model, prover_model_path, scales, "resources", settings_path)

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_43756/589984064.py:9: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return s.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_43756/589984064.py:9: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return s.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))


==== Generate & Calibrate Setting ====




 <------------- Numerical Fidelity Report (input_scale: 8, param_scale: 8, scale_input_multiplier: 10) ------------->

+----------------+---------------+-----------+---------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error     | median_error  | max_error | min_error     | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+----------------+---------------+-----------+---------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| -0.00078201294 | -0.0015640259 | 0         | -0.0015640259 | 0.00078201294  | 0.0015640259     | 0.0015640259  | 0             | 0.0000012230885    | -0.00001584626     | 0.00001584626          |
+----------------+---------------+-----------+---------------+----------------+--------

scale:  [8]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":8,"param_scale":8,"scale_rebase_multiplier":10,"lookup_range":[-25268,25754],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":20992,"total_assignments":12490,"total_const_size":1222,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,8],"model_input_scales":[8,8],"module_sizes":{"kzg":[],"poseidon":[20992,[2]]},"required_lookups":[{"GreaterThan":{"a":0.0}}],"required_range_checks":[],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1709450741807}


In [9]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 5.530666828155518 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 49.3515625
==== Generating Proof ====
proof:  {'instances': [['c2b6968dfce66ece687a00d15dea969a5dab54b503c9ac9d0ad01308cedbb32e', 'dfb9f22d8e9142be6a825acd931b1e936e0f99940f77141cdc735127c1c44016', '0100000000000000000000000000000000000000000000000000000000000000', '5a31000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x21906659bad4d0bd1f21c493dd16dc615fe51ce0ad39f73b32c680b4ff7dcf17099dfaa9ded6e9281e79ecf93612f100cc3ac175f9ad79a2e17dee5e523bb6a4097d2c8960e34e36a7b229113d4e0dbe7893fa1064063298e0af499186f0f165128e6901be7a964d76324498ebd51c5513243b0f6abeda7326f674d41c5999e12cd739ac2d52481d28d0bd89699f9ea2b23e6722173458ac285120f0a72ece012dadd5b316451ee9b4e8e5bf7719b870564594369b522df202f1c4900092e4252ee16339d9058db1a5b8b3ae344d935ec6458267cc736fe4163442c4595677d51eb402d0e767e2c2a32ed5bb4edd1ee1ac306f499783c6010ee50fa0d86353191287b5

In [10]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: [49.3515625]
